# Data Lakes with AWS Glue

## Overview
1. Tạo S3 bucket
2. Tạo EC2 VPC endpoint
3. Tạo IAM role -> Cấp policy S3, Glue
4. Trong AWS CLI -> Clone 1 repo về -> Copy 1 file `.json` vào S3 bucket
5. Vào Glue Tạo ETL job, mđ là load file `.json` trên từ S3, filter, rồi lưu file `.json` đã filter vào S3 (có dùng IAM role) -> Chạy ETL job

### C4L4.6 - Setting up an S3 VPC Gateway Endpoint
1. Vào CloudShell
2. Tạo một bucket mới
    ```bash
    aws s3 mb s3://stedi-lake-house-hieu
    ```
    Vào bucket vừa tạo -> **Permissions**. Ở **Block public access**, ấn **Edit** -> Bỏ tick **Block all public access** -> **Save**

    Xem có gì trong bucket
    ```bash
    aws s3 ls s3://stedi-lake-house-hieu
    ```

3. Xem ID của VPC
    ```bash
    aws ec2 describe-vpcs
    ```

    Xem `“VpcId”`.

4. Tìm route table
    ```bash
    aws ec2 describe-route-tables
    ```

    Xem `“RouteTableId”` và `“VpcId”`.

5. Tạo VPC endpoint

    ```bash
    aws ec2 create-vpc-endpoint --vpc-id <VpcId> --service-name com.amazonaws.us-east-1.s3
    --route-table-ids <RouteTableId>
    ```

### C4L4.8 - Creating the Glue Service Role
Chú ý: Nhớ đổi tên S3 bucket ở các lệnh dưới theo bucket đã tạo.
1. Tạo một IAM role
    ```bash
    aws iam create-role --role-name my-glue-service-role --assume-role-policy-document '{
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "glue.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }'
    ```

2. Cho policy vào role vừa tạo (nhớ sửa 2 chỗ Resource là S3 bucket đã tạo)
    ```bash
    aws iam put-role-policy --role-name my-glue-service-role --policy-name S3Access --policy-document '{
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "ListObjectsInBucket",
                "Effect": "Allow",
                "Action": [
                    "s3:ListBucket"
                ],
                "Resource": [
                    "arn:aws:s3:::stedi-lake-house-hieu"
                ]
            },
            {
                "Sid": "AllObjectActions",
                "Effect": "Allow",
                "Action": "s3:*Object",
                "Resource": [
                    "arn:aws:s3:::stedi-lake-house-hieu/*"
                ]
            }
        ]
    }'
    ```

3. Cấp Glue Privileges cho role trên
    ```bash
    aws iam put-role-policy --role-name my-glue-service-role --policy-name GlueAccess --policy-document '{
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "glue:*",
                    "s3:GetBucketLocation",
                    "s3:ListBucket",
                    "s3:ListAllMyBuckets",
                    "s3:GetBucketAcl",
                    "ec2:DescribeVpcEndpoints",
                    "ec2:DescribeRouteTables",
                    "ec2:CreateNetworkInterface",
                    "ec2:DeleteNetworkInterface",
                    "ec2:DescribeNetworkInterfaces",
                    "ec2:DescribeSecurityGroups",
                    "ec2:DescribeSubnets",
                    "ec2:DescribeVpcAttribute",
                    "iam:ListRolePolicies",
                    "iam:GetRole",
                    "iam:GetRolePolicy",
                    "cloudwatch:PutMetricData"
                ],
                "Resource": [
                    "*"
                ]
            },
            {
                "Effect": "Allow",
                "Action": [
                    "s3:CreateBucket",
                    "s3:PutBucketPublicAccessBlock"
                ],
                "Resource": [
                    "arn:aws:s3:::aws-glue-*"
                ]
            },
            {
                "Effect": "Allow",
                "Action": [
                    "s3:GetObject",
                    "s3:PutObject",
                    "s3:DeleteObject"
                ],
                "Resource": [
                    "arn:aws:s3:::aws-glue-*/*",
                    "arn:aws:s3:::*/*aws-glue-*/*"
                ]
            },
            {
                "Effect": "Allow",
                "Action": [
                    "s3:GetObject"
                ],
                "Resource": [
                    "arn:aws:s3:::crawler-public*",
                    "arn:aws:s3:::aws-glue-*"
                ]
            },
            {
                "Effect": "Allow",
                "Action": [
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents",
                    "logs:AssociateKmsKey"
                ],
                "Resource": [
                    "arn:aws:logs:*:*:/aws-glue/*"
                ]
            },
            {
                "Effect": "Allow",
                "Action": [
                    "ec2:CreateTags",
                    "ec2:DeleteTags"
                ],
                "Condition": {
                    "ForAllValues:StringEquals": {
                        "aws:TagKeys": [
                            "aws-glue-service-resource"
                        ]
                    }
                },
                "Resource": [
                    "arn:aws:ec2:*:*:network-interface/*",
                    "arn:aws:ec2:*:*:security-group/*",
                    "arn:aws:ec2:*:*:instance/*"
                ]
            }
        ]
    }'
    ```

### C4L4.11 - Define a Trusted Zone Path

1. Clone repo
    ```bash
    git clone https://github.com/udacity/nd027-Data-Engineering-Data-Lakes-AWS-Exercises.git
    cd nd027-Data-Engineering-Data-Lakes-AWS-Exercises
    cd project/starter/customer/landing
    ```

    Tại đây có một file `customer-1691348231425.json`.

2. Copy file trên vào S3
    ```bash
    aws s3 cp ./customer-1691348231425.json s3://stedi-lake-house-hieu/customer/landing/
    ```

    Kiếm tra bucket:
    ```bash
    aws s3 ls s3://stedi-lake-house-hieu/customer/landing/
    ```

    Để xóa file:
    ```bash
    aws s3 rm s3://stedi-lake-house-hieu/customer/landing/customer-1691348231425.json
    ```

3. Vào **AWS Glue -> ETL jobs** -> **Visual ETL** -> **Create job**
4. Chọn **Source** là `Amazon S3`, **Transform** là `Filter`, **Target** là `Amazon S3`.

    <img src="images/aws1.png" width=1500>

5. Ấn vào **Data source** trên đồ thị, chọn **S3 source type** là **S3 location**, điền **S3 URL** là `s3://stedi-lake-house-hieu/customer/landing/`.
Chọn **Data format** là `JSON`. Ấn vào **Inter schema**, kiếm tra schema trong **Output schema**.

    <img src="images/aws2.png" width=1500>

6. Ấn vào **Transform** trên đồ thị, đổi tên thành `PrivacyFilter`.
Chọn **Filter** là **Global AND**. Ấn vào Add condition, chọn **Key** là `shareWithResearchAsOfDate`, **Operation** là `!=`, **Value** là `0`.

    <img src="images/aws3.png" width=1500>

7. Ấn vào **Data target** trên đồ thị, đổi tên thành `Trusted Customer Zone`.
Chọn **Format** là `JSON`, **Compression Type** là `None`.
Điền **S3 Target Location** là `s3://stedi-lake-house-hieu/customer/trusted/`.

    <img src="images/aws4.png" width=1500>

8. Ấn vào **Job details**, đặt **Name** là `Customer Landing to Trusted`.
Chọn **IAM Role** là `my-glue-service-role`.
Còn lại để nguyên, ấn vào **Save** rồi ấn **Run**.

    <img src="images/aws5.png" width=700>

9. Vào **ETL jobs**, ấn vào **Customer Landing to Trusted** -> **Runs** -> Vào S3 kiểm tra

    <img src="images/aws6.png" width=1500>

10. Vào **ETL jobs** -> **Job run monitoring**. Tick **Customer Landing to Trusted**, ấn **View run details**.

    <img src="images/aws7.png" width=1500>

    Ấn vào **Error logs** để kiểm tra (sẽ vào trong **Management and Governance** -> **CloudWatch**).

    <img src="images/aws8.png" width=1500>